<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/%EB%B0%A9%ED%95%99%EC%84%B8%EB%AF%B8%EB%82%98_%EB%AA%A8%EB%8D%B8%EB%A7%81_0813.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

#### 1. 데이터 불러오기

In [5]:
train = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/Train_data.csv", engine = "python")
# train.info()
# test = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/Test_data.csv")

#### 2. 시간 변수 변형

In [8]:
train["event_datetime"] = pd.to_datetime(train["event_datetime"])
train["hour"] = train["event_datetime"].dt.hour
train["hour"] = train["hour"].astype(str)

#### 3. 불필요한 변수 제거 

In [9]:
delete_col = ["bid_id", "device_ifa", "device_country", "event_datetime"]
train_1 = train.drop(delete_col, axis = 1)
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2725271 entries, 0 to 2725270
Data columns (total 22 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   click                   int64 
 1   ssp_id                  object
 2   campaign_id             object
 3   adset_id                object
 4   placement_type          object
 5   media_id                object
 6   media_name              object
 7   media_bundle            object
 8   media_domain            object
 9   publisher_id            object
 10  publisher_name          object
 11  device_os               object
 12  device_os_version       object
 13  device_model            object
 14  device_carrier          object
 15  device_make             object
 16  device_connection_type  object
 17  device_language         object
 18  device_region           object
 19  device_city             object
 20  advertisement_id        object
 21  hour                    object
dtypes: int64(1), objec

In [11]:
Y = train_1["click"]
X = train_1.drop("click",axis = 1)

### One Hot Encoding + Random Forest

#### 4. X와 Y분리

#### 5. 전체 데이터 중 10%만 사용하기 

In [ ]:
X_use, X_rest, Y_use, Y_rest = train_test_split(X, Y, test_size=0.9, train_size=0.1, stratify= Y)
# X_train, X_test, y_train, y_test = train_test_split(X_rest, y_rest, test_size=0.25, train_size=0.75, stratify='column')

#### 6. Cross Validation 하기

In [ ]:
encoder_X  = OneHotEncoder()
encoder_X.fit(X_use)
X_ohe = encoder_X.transform(X_use)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 200, random_state = 1234)
kfold = KFold(n_splits=5, shuffle = True, random_state = 1234)
scores = cross_val_score(rf, X_ohe, Y_use, cv=kfold, scoring = "neg_log_loss", verbose = 1)
print(f"랜포의 성능은: {scores}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


1000층까지 가지를 내린 랜포의 성능은: [-0.40653773 -0.41724055 -0.41375715 -0.39974547 -0.41037066]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 255.6min finished


## One Hot Encoding + XGBoost

### 1. XGBoost 설치

In [ ]:
!pip install xgboost

In [12]:
import xgboost as xgb

### 1. X와 Y 분리 

In [13]:
Y = train_1["click"]
X = train_1.drop("click",axis = 1)

### 2. 전체 데이터 중 10%만 사용하기 

In [14]:
X_use, X_rest, Y_use, Y_rest = train_test_split(X, Y, test_size=0.9, train_size=0.1, stratify= Y)
# X_train, X_test, y_train, y_test = train_test_split(X_rest, y_rest, test_size=0.25, train_size=0.75, stratify='column')

### 3. One Hot Encoding 하기 

In [15]:
encoder_X  = OneHotEncoder()
encoder_X.fit(X_use)
X_ohe = encoder_X.transform(X_use)

### 4. Cross Validaion 하기 

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
boost = xgb.XGBClassifier(learning_rate = 0.1,
                          objective= "binary:logistic", 
                          silent = False)
kfold = KFold(n_splits=5, shuffle = True, random_state = 1234)
scores = cross_val_score(boost, X_ohe, Y_use, cv=kfold, scoring = "neg_log_loss", verbose = 1)
print(f"XGBoost의 성능은: {scores}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


XGBoost의 성능은: [-0.23697203 -0.24078358 -0.23883074 -0.23467519 -0.24555165]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min finished


## Target Encoding + One Hot Encoding + XGBoost
1.   Mean Encoding : "adst_id", "media_id", "media_name", "media_bundle", "media_domain", "publisher_id", "publisher_name", "device_os_version", "device_model", "devcie_carrier", "device_make", "device_region", "device_city"



In [85]:
!pip install category_encoders


     |████████████████████████████████| 81kB 3.9MB/s 


In [18]:
X_use, _, Y_use, _ = train_test_split(X, Y, test_size=0.9, train_size=0.1, stratify= Y)

### 2. 각 인코딩 별 변수 나누기 

In [21]:
col = X_use.columns
col_mean = [ "adset_id", "media_id", "media_name", "media_bundle", "media_domain", "publisher_id", "publisher_name", "device_os_version", "device_model", "device_carrier", "device_make", "device_region", "device_city"]
col_ohe = col[~col.isin(col_mean)]

In [22]:
X_ohe_before = X_use[col_ohe]
X_mean_before = X_use[col_mean]

In [23]:
print(X_use.shape)
print(Y_use.shape)
print(X_mean_before.shape)

(272527, 21)
(272527,)
(272527, 13)


### 3. One Hot Encoding

In [24]:
# train_ohe_before, train_mean_before, Y_use

In [25]:
encoder_ohe  = OneHotEncoder()
encoder_ohe.fit(X_ohe_before)
X_ohe = encoder_ohe.transform(X_ohe_before).toarray()

### 4, Mean Encoding

In [28]:
import category_encoders as ce
import xgboost as xgb
folds = StratifiedKFold(random_state = 1234)
encoder_target = ce.target_encoder.TargetEncoder()

score = []
for train_idx, val_idx in folds.split(X_mean_before, Y_use):
    print(" 데이터 불러오기 및 train/validation 나누기") 
    x_train_mean_before = X_mean_before.iloc[train_idx, :]
    x_val_mean_before = X_mean_before.iloc[val_idx]

    x_train_ohe = X_ohe[train_idx]
    x_val_ohe = X_ohe[val_idx]

    y_train = Y_use.iloc[train_idx]
    y_val = Y_use.iloc[val_idx]

    print("타겟 인코더 피팅")
    encoder_target.fit(x_train_mean_before, y_train)

    print("인코딩")
    x_train_mean = encoder_target.transform(x_train_mean_before)
    x_val_mean = encoder_target.transform(x_val_mean_before)

    x_train = np.hstack((x_train_mean, x_train_ohe))
    x_val = np.hstack((x_val_mean, x_val_ohe))

    boost = xgb.XGBClassifier(learning_rate = 0.1,
                          objective= "binary:logistic", 
                          silent = False)
    boost.fit(x_train, y_train)

    pred = boost.predict_proba(x_val)

    score_tmp = log_loss(y_val, pred)
    print(score_tmp)

    print("끝")
 [-0.23697203 -0.24078358 -0.23883074 -0.23467519 -0.24555165]
    

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 데이터 불러오기 및 train/validation 나누기
타겟 인코더 피팅
인코딩
0.23671239138828915
끝
 데이터 불러오기 및 train/validation 나누기
타겟 인코더 피팅
인코딩
0.24141921672124841
끝
 데이터 불러오기 및 train/validation 나누기
타겟 인코더 피팅
인코딩
0.23788788779929426
끝
 데이터 불러오기 및 train/validation 나누기
타겟 인코더 피팅
인코딩
0.2386395166997626
끝
 데이터 불러오기 및 train/validation 나누기
타겟 인코더 피팅
인코딩
0.23794808290324312
끝


In [31]:
print(np.mean([-0.23697203, -0.24078358, -0.23883074, -0.23467519, -0.24555165]))
print(np.mean([0.23671239138828915,0.24141921672124841,0.23788788779929426, 0.2386395166997626, 0.23794808290324312]))

-0.23936263800000002
0.23852141910236752


### 2. 

In [ ]:
# #### importance plot을 그려보려 했으나 변수가 너무 많아 유의미하지 않음 

# Importances = rf.feature_importances_


# std = np.std([tree.feature_importances_ for tree in rf.estimators_],
#              axis=0)
# indices = np.argsort(Importances)[::-1]

# # Print the feature ranking
# print("Feature ranking:")

# for f in range(X_ohe.shape[1]):
#     print("%d. feature %d (%f)" % (f + 1, indices[f], Importances[indices[f]]))

# # Plot the impurity-based feature importances of the forest
# plt.figure()
# plt.title("Feature importances")
# plt.bar(range(X_ohe.shape[1]), Importances[indices],
#         color="r", yerr=std[indices], align="center")
# plt.xticks(range(X_ohe.shape[1]), indices)
# plt.xlim([-1, X_ohe.shape[1]])
# plt.show()

